# Acquire and Prep - Wrangle

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
from env import host, user, password

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# function to query database and return zillow df
def get_data_from_sql():
    query = """
    SELECT bedroomcnt as bedrooms, bathroomcnt as bathrooms, calculatedfinishedsquarefeet as square_feet, 
    taxvaluedollarcnt, yearbuilt as year_built, taxamount taxes,  fips 
    FROM properties_2017 WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279;
    """
    df = pd.read_sql(query, get_connection('zillow'))
    return df

In [4]:
df = get_data_from_sql()

In [5]:
df.head()

,bedrooms,bathrooms,square_feet,taxvaluedollarcnt,year_built,taxes,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [6]:
df.shape


(2152864, 7)

In [7]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
bedrooms,2152853.0,3.287195,0.954757,0.00,3.0000,3.000,4.0000,25.00
bathrooms,2152853.0,2.230687,0.999281,0.00,2.0000,2.000,3.0000,32.00
square_feet,2144379.0,1862.855178,1222.125124,1.00,1257.0000,1623.000,2208.0000,952576.00
taxvaluedollarcnt,2152371.0,461896.052361,699675.940049,1.00,188170.0000,327671.000,534527.0000,98428909.00
year_built,2143526.0,1960.949681,22.162196,1801.00,1949.0000,1958.000,1976.0000,2016.00
taxes,2148422.0,5634.863752,8178.908996,1.85,2534.9725,4108.945,6414.3175,1337755.86
fips,2152864.0,6048.377330,20.433289,6037.00,6037.0000,6037.000,6059.0000,6111.00


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152864 entries, 0 to 2152863
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   bedrooms           float64
 1   bathrooms          float64
 2   square_feet        float64
 3   taxvaluedollarcnt  float64
 4   year_built         float64
 5   taxes              float64
 6   fips               float64
dtypes: float64(7)
memory usage: 115.0 MB


In [9]:
df.value_counts()

bedrooms  bathrooms  square_feet  taxvaluedollarcnt  year_built  taxes     fips  
3.0       1.0        1002.0       39230.0            1962.0      419.14    6111.0    21
2.0       2.5        1008.0       101575.0           1988.0      1450.58   6059.0    16
4.0       3.0        2539.0       678000.0           2005.0      9094.20   6059.0    14
3.0       2.5        1358.0       194027.0           2000.0      2317.92   6111.0    14
                     2060.0       455635.0           2007.0      5271.34   6059.0    13
                                                                                     ..
          2.0        1312.0       55069.0            1954.0      874.62    6059.0     1
                                  54603.0            1963.0      857.48    6059.0     1
                                  54300.0            1955.0      954.44    6037.0     1
                                  54293.0            1956.0      947.47    6037.0     1
25.0      20.0       11700.0      1608

## Prepare the Data

In [10]:
# Find the total number of Null values in each column of our DataFrame.

df.isnull().sum()

bedrooms               11
bathrooms              11
square_feet          8485
taxvaluedollarcnt     493
year_built           9338
taxes                4442
fips                    0
dtype: int64

In [11]:
# Check for any Null values in each column of our DataFrame.

df.isnull().any()


bedrooms              True
bathrooms             True
square_feet           True
taxvaluedollarcnt     True
year_built            True
taxes                 True
fips                 False
dtype: bool

In [12]:
# Return the names for any columns in our DataFrame with any Null values.

df.columns[df.isnull().any()]


Index(['bedrooms', 'bathrooms', 'square_feet', 'taxvaluedollarcnt',
       'year_built', 'taxes'],
      dtype='object')

Finding Odd Values

Let's find the odd value in exam3 that is causing this numeric column to be coerced into an object data type.

In [13]:
df.bedrooms.value_counts(dropna=False, ascending=True)


25.0         1
16.0         2
18.0         3
15.0         6
14.0         7
NaN         11
12.0        12
13.0        16
11.0        34
10.0       121
9.0        291
8.0       1107
7.0       4807
0.0      13188
1.0      23166
6.0      25166
5.0     150866
2.0     335473
4.0     634289
3.0     964298
Name: bedrooms, dtype: int64

In [14]:
# check value counts for bedrooms column

df['bedrooms'].value_counts(ascending=False)

3.0     964298
4.0     634289
2.0     335473
5.0     150866
6.0      25166
1.0      23166
0.0      13188
7.0       4807
8.0       1107
9.0        291
10.0       121
11.0        34
13.0        16
12.0        12
14.0         7
15.0         6
18.0         3
16.0         2
25.0         1
Name: bedrooms, dtype: int64

In [15]:
#check value counts for bathrooms column

df['bathrooms'].value_counts(ascending=True)


32.00         1
19.50         1
19.00         1
14.50         1
1.75          3
11.50         3
12.50         3
17.00         4
20.00         6
18.00         8
16.00        12
10.50        14
0.50         16
15.00        17
14.00        25
13.00        39
9.50         50
12.00        73
8.50        110
11.00       146
10.00       325
7.50        384
9.00        713
6.50       1333
8.00       1692
7.00       4394
5.50       6217
6.00      10747
0.00      13028
4.50      19506
5.00      28362
3.50      28518
1.50      31211
4.00      82155
2.50     142981
1.00     414324
3.00     422841
2.00     943589
Name: bathrooms, dtype: int64

In [16]:
# check value counts for taxes column
df['taxes'].value_counts(ascending=True)

93164.54      1
36677.65      1
1885.16       1
5248.31       1
9314.30       1
           ... 
49.18        46
285.30       67
343.52       75
344.18       77
345.72      172
Name: taxes, Length: 918838, dtype: int64

In [17]:
# Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df.

df = df.replace(r'^\s*$', np.nan, regex=True)

In [18]:
# Now .info() shows us that bedrooms has a Null value instead of a whitespace disguised as a non-null value.

df.isnull().any()

bedrooms              True
bathrooms             True
square_feet           True
taxvaluedollarcnt     True
year_built            True
taxes                 True
fips                 False
dtype: bool